### Zigbang 원룸 매물 데이터 수집

In [1]:
import requests
import pandas as pd

#### Process

- 동이름으로 위도 경도 구하기  
- 위도 경도로 geohash 알아내기  
- geohash로 매물 아이디 가져오기  
- 매물 아이디로 매물 정보 가져오기  

#### 1. 동이름으로 위도 경도 구하기

In [42]:
addr = '성내동'
url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'

response = requests.get(url)
response

<Response [200]>

In [10]:
data = response.json()['items'][5]
lat, lon = data['lat'], data['lng']
lat, lon

(37.529273986816406, 127.1300277709961)

#### 2. 위도 경도로 geohash 알아내기

In [11]:
!pip install geohash2 --quiet

In [12]:
import geohash2

In [15]:
geohash = geohash2.encode(lat, lon, precision=5) # precision이 높아질수록 작은 영역에 대한 geohash값을 알려줌. (고해상도)
geohash

'wydmk'

#### 3. geohash로 매물 아이디 가져오기

In [16]:
url = f'https://apis.zigbang.com/v2/items/oneroom?\
geohash={geohash}&depositMin=0&rentMin=0&salesTypes[0]=전세&salesTypes[1]=월세&domain=zigbang&checkAnyItemWithoutFilter=true'

response = requests.get(url)
response

<Response [200]>

In [20]:
item_ids = [data['itemId'] for data in response.json()['items']]
len(item_ids), item_ids[:5]

(1209, [42195372, 42213938, 42132177, 42193550, 42234254])

#### 4. 매물 아이디로 매물 정보 가져오기

In [28]:
# 999개까지만 가져와진다.
url = 'https://apis.zigbang.com/v2/items/list'

params = {
    'doamin': 'zigbang',
    'item_ids': item_ids[:900]
}

response = requests.post(url, data=params)
response

<Response [200]>

In [29]:
data = response.json()['items']
df = pd.DataFrame(data)[['item_id', 'sales_title', 'deposit', 'rent', 'size_m2', 'floor', 'building_floor', 'title', 'manage_cost', 'address1']]
df.tail(2)

,item_id,sales_title,deposit,rent,size_m2,floor,building_floor,title,manage_cost,address1
898,42126262,월세,2000,170,43.76,7,17,사업자 빠른입주가능 1년차 신축,15,서울시 송파구 방이동
899,42165806,월세,3000,150,43.86,5,17,사업자OK 1.5룸 신축 고층 건조기 스타일러,9,서울시 송파구 방이동


In [44]:
# 5. function

def get_address(addr):
    url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'

    response = requests.get(url)
    data = response.json()['items'][5] #서울시 강동구 성내동
    return data['lat'], data['lng']

def get_itemId(lat, lon):
    # get itemId
    geohash = geohash2.encode(lat, lon, precision=6) # precision이 높아질수록 작은 영역에 대한 geohash값을 알려줌. (고해상도)

    url = f'https://apis.zigbang.com/v2/items/oneroom?geohash={geohash}&depositMin=0&rentMin=0&salesTypes[0]=전세&salesTypes[1]=월세&domain=zigbang&checkAnyItemWithoutFilter=true'
    response = requests.get(url)
    
    return [data['itemId'] for data in response.json()['items']]
    
def get_data(item_ids):
    # get data
    url = 'https://apis.zigbang.com/v2/items/list'

    params = {
        'doamin': 'zigbang',
        'item_ids': item_ids[:900]
    }

    response = requests.post(url, data=params)

    data = response.json()['items']
    return pd.DataFrame(data)[['item_id', 'sales_title', 'deposit', 'rent', 'size_m2', 'floor', 'building_floor', 'title', 'manage_cost', 'address1']]

def crawling_zigbang(addr):
    lat, lon = get_address(addr)
    item_ids = get_itemId(lat, lon)
    df = get_data(item_ids)
    return df

In [45]:
df = crawling_zigbang("성내동")
df.tail(5)

,item_id,sales_title,deposit,rent,size_m2,floor,building_floor,title,manage_cost,address1
48,41713863,월세,2000,55,25.00,2,5,"초역세권 둔촌동역 5분, 신축 원룸, 풀옵션, 초가성비 매물",7,서울시 강동구 성내동
49,42118309,월세,3000,50,19.83,2,5,V둔촌동역도보5분V신축급입주풀옵션원룸V최상의룸컨디션V,7,서울시 강동구 성내동
50,42170911,월세,2000,55,17.90,2,5,신축깨끗한 풀옵션원룸.내부깨끗하고 수납좋음 둔촌역가까워요,7,서울시 강동구 성내동
51,42177783,월세,3000,50,19.80,2,5,신축급 넓은 원룸 가성비굿,7,서울시 강동구 성내동
52,42232645,월세,3000,50,16.53,2,5,둔촌동역. 2년차신축. 풀옵션.,7,서울시 강동구 성내동


In [46]:
df.head(5)

,item_id,sales_title,deposit,rent,size_m2,floor,building_floor,title,manage_cost,address1
0,42032346,월세,5000,80,57.00,4,4,5층탑층 주인거주 올수리 깨끗한 투룸 지정주차,3,서울시 강동구 성내동
1,42172753,월세,300,25,13.22,5,5,잠만잘수있는 옥탑방,3,서울시 강동구 성내동
2,42216496,전세,30000,0,75.01,3,5,"JYP엔터 인근, 올림픽공원 3분, 넓고 귀한 쓰리룸, 올수리 앤틱하우스.",3,서울시 강동구 성내동
3,42226308,전세,11000,0,39.67,반지하,3,"LH, SH, HUG버팀목 가능. 올림픽공원 도보 3분.",5,서울시 강동구 성내동
4,42215154,전세,4000,0,30.00,반지하,3,둔촌동역 역세권 단독 바닥층 2룸 전세,0,서울시 강동구 성내동


In [47]:
df['address1'].unique()

array(['서울시 강동구 성내동'], dtype=object)

<PreparedRequest [GET]>